In [1]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import h5py
from math import pi

from analysis.config import birds, h5_path_dict, pickle_dir
from analysis.ExpData import ExpData
from analysis.PlaceCalc import PlaceCalc

In [3]:
data = {}
place_calc = PlaceCalc(
    num_shuffles=200, threshold=0.99, dimension=1, bins_2d=None
    )
for key in h5_path_dict.keys():
    for fil in range(len(h5_path_dict[key])):
        fil_string = h5_path_dict[key][fil].as_posix()
        f = h5py.File(fil_string, 'r')
        exp_data = ExpData(f)
        data[fil_string] = {}
        spatial_info, significance_1d = place_calc.calc_place_cells(exp_data)
        data[fil_string]['spatial_info'] = spatial_info
        data[fil_string]['significance'] = significance_1d
with open(pickle_dir / 'place_cells_1d.p', 'wb') as f: 
    pickle.dump(data, f)

/home/chingf/Code/cache-arena-analysis/analysis/tuning_measures/mutual_info.py:20: RuntimeWarning: divide by zero encountered in log2
  log_term = np.log2(condn_mean_fr/mean_fr)
/home/chingf/Code/cache-arena-analysis/analysis/tuning_measures/mutual_info.py:20: RuntimeWarning: invalid value encountered in true_divide
  log_term = np.log2(condn_mean_fr/mean_fr)
/home/chingf/Code/cache-arena-analysis/analysis/PlaceCalc.py:33: RuntimeWarning: invalid value encountered in true_divide
  spatial_info /= shuffled_spatial_info
